In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

def setup_save_directory():
    """
    ユーザーとの対話を通じて、画像の保存先ディレクトリを設定する関数。
    - 新規作成: 親ディレクトリと新規ディレクトリ名を指定して作成する。
    - 既存利用: 既存のディレクトリパスを指定する。
    Returns:
        str: 有効な保存先ディレクトリのパス。
    """
    while True:
        choice = input("保存先ディレクトリを選択してください (1: 新規作成, 2: 既存のディレクトリを指定): ")

        if choice == '1':
            # --- 新しいディレクトリを作成する場合 ---
            while True:
                parent_dir = input("\n新しいディレクトリを作成する場所（親ディレクトリ）のパスを入力してください (例: D:\\images): ")
                if os.path.isdir(parent_dir):
                    break
                else:
                    print(f"エラー: '{parent_dir}' は存在しないか、ディレクトリではありません。有効なパスを入力してください。")
            
            new_dir_name = input(f"'{parent_dir}' 内に作成する新しいディレクトリ名を入力してください (例: ap_siken_images): ")
            if not new_dir_name:
                print("エラー: ディレクトリ名は空にできません。")
                continue
            
            full_path = os.path.join(parent_dir, new_dir_name)
            try:
                os.makedirs(full_path, exist_ok=True)
                print(f"-> ディレクトリ '{full_path}' を作成しました。")
                return full_path
            except OSError as e:
                print(f"エラー: ディレクトリの作成に失敗しました。 {e}")
                continue

        elif choice == '2':
            # --- 既存のディレクトリを指定する場合 ---
            while True:
                existing_dir = input("\n保存先となる既存のディレクトリのパスを入力してください (例: D:\\steganography\\manga): ")
                if os.path.isdir(existing_dir):
                    print(f"-> 既存のディレクトリ '{existing_dir}' を使用します。")
                    return existing_dir
                else:
                    print(f"エラー: '{existing_dir}' は存在しないか、ディレクトリではありません。有効なパスを入力してください。")
        
        else:
            print("エラー: '1' または '2' を入力してください。")

def select_save_format():
    """
    ユーザーに画像の保存形式を選択させる関数。
    Returns:
        str: 選択されたファイル形式の拡張子 (例: "png")
    """
    while True:
        # ドットなしで入力させる
        file_format = input("保存するファイル形式を選択してください (png, jpg, jpeg): ").lower()
        if file_format in ["png", "jpg", "jpeg"]:
            print(f"-> ファイル形式として '.{file_format}' を選択しました。")
            return file_format
        else:
            print("エラー: 'png', 'jpg', 'jpeg' のいずれかを入力してください。")

def main():
    """
    スクレイピング処理を実行するメイン関数。
    """
    # 1. 対象のWebページ
    TARGET_URL = "https://www.ap-siken.com/apkakomon.php"  # 任意の画像を含むページに変更する

    # 2. 保存先ディレクトリとファイル形式をユーザーに選択・設定させる
    print("--- 画像の保存先設定 ---")
    SAVE_DIR = setup_save_directory()
    SAVE_FORMAT = select_save_format()
    print("------------------------\n")

    # 3. HTML取得
    print(f"'{TARGET_URL}' からHTMLを取得しています...")
    try:
        response = requests.get(TARGET_URL)
        response.raise_for_status()  # ステータスコードが200番台以外の場合は例外を発生させる
    except requests.exceptions.RequestException as e:
        print(f"エラー: Webページの取得に失敗しました。 {e}")
        return # 関数を終了

    # 4. HTML解析と画像タグの取得
    soup = BeautifulSoup(response.content, "html.parser")
    img_tags = soup.find_all("img")
    
    if not img_tags:
        print("ページ内に画像が見つかりませんでした。")
        return

    print(f"{len(img_tags)} 個の画像が見つかりました。ダウンロードを開始します。")

    # 5. 画像URLごとに処理
    saved_count = 0
    for i, img in enumerate(img_tags, start=1):
        img_url = img.get("src")
        if not img_url:
            print(f"警告: {i}番目のimgタグにはsrc属性がありませんでした。スキップします。")
            continue

        # 相対URLを絶対URLに変換
        img_url_absolute = urljoin(TARGET_URL, img_url)

        # 画像データ取得と保存
        try:
            print(f"({i}/{len(img_tags)}) ダウンロード中: {img_url_absolute}")
            img_data = requests.get(img_url_absolute, timeout=10).content
            
            # URLからファイル名を推測し、拡張子を取り除く
            filename_from_url = os.path.basename(urljoin(img_url_absolute, '.'))
            filename_from_url = filename_from_url.split('?')[0] # クエリ文字列を削除
            base_name, _ = os.path.splitext(filename_from_url)
            
            if base_name:
                # 連番、元のファイル名（拡張子なし）、選択された拡張子でファイル名を作成
                filename = f"{i:03d}_{base_name}.{SAVE_FORMAT}"
            else:
                # URLからファイル名が取得できない場合は連番と選択された拡張子を振る
                filename = f"image_{i:03d}.{SAVE_FORMAT}"

            save_path = os.path.join(SAVE_DIR, filename)

            # 書き込み
            with open(save_path, "wb") as f:
                f.write(img_data)
            saved_count += 1
            
            # サーバーに負荷をかけすぎないための待機時間
            time.sleep(0.5)

        except requests.exceptions.RequestException as e:
            print(f"エラー: {img_url_absolute} のダウンロードに失敗しました: {e}")
        except Exception as e:
            print(f"エラー: {img_url_absolute} の保存中に予期せぬエラーが発生しました: {e}")

    print(f"\n完了しました。{saved_count}個の画像を'{SAVE_DIR}'に保存しました。")

if __name__ == "__main__":
    main()


--- 画像の保存先設定 ---
エラー: '1' または '2' を入力してください。
エラー: '1' または '2' を入力してください。
エラー: '1' または '2' を入力してください。
エラー: '1' または '2' を入力してください。
エラー: '1' または '2' を入力してください。
エラー: '1' または '2' を入力してください。
エラー: '1' または '2' を入力してください。
エラー: '1' または '2' を入力してください。
エラー: '1' または '2' を入力してください。
エラー: '1' または '2' を入力してください。
エラー: '1' または '2' を入力してください。
エラー: '1' または '2' を入力してください。
